## 晟虹考勤核算项目

In [8]:
from openpyxl import load_workbook
from itertools import dropwhile
import datetime
from typing import List, Union

# 加载 Excel 文件
wb = load_workbook(r'D:\DATA\code\PythonCode\openpyxl项目实战-考勤计算\5555.xlsx')
sheet = wb['Sheet2']
sheet

<Worksheet "Sheet2">

In [11]:
def process_attendance_records(records: List[Union[str, datetime.time]]) -> List[Union[str, datetime.time]]:
    """
    处理打卡记录，按照规则筛选有效打卡时间

    Args:
        records: 原始打卡记录列表，格式为 [工号, 姓名, 公司, 日期, 时间1, 时间2, ...]

    Returns:
        处理后的打卡记录列表，包含筛选后的有效打卡时间
    """
    # 提取时间部分（从索引4开始）
    times = records[4:]

    if not times:
        return records

    # 早上上班：取第一次打卡
    morning_start = times[0]

    # 早上下班：最接近11:30且大于11:30
    target_am_end = datetime.time(11, 30)
    am_end_candidates = [t for t in times if t > target_am_end]
    am_end = min(am_end_candidates,
                 key=lambda t: (datetime.datetime.combine(datetime.date.today(), t) -
                                datetime.datetime.combine(datetime.date.today(),
                                                          target_am_end)).total_seconds()) if am_end_candidates else None

    # 下午上班：最接近13:00且小于13:00（在早上下班之后）
    target_pm_start = datetime.time(13, 0)
    pm_start_candidates = [t for t in times if t < target_pm_start and (am_end is None or t > am_end)]
    pm_start = max(pm_start_candidates) if pm_start_candidates else None

    # 下午下班：最接近17:00且大于17:00（在下午上班之后）
    target_pm_end = datetime.time(17, 0)
    pm_end_candidates = [t for t in times if t > target_pm_end and (pm_start is None or t > pm_start)]
    pm_end = min(pm_end_candidates,
                 key=lambda t: (datetime.datetime.combine(datetime.date.today(), t) -
                                datetime.datetime.combine(datetime.date.today(),
                                                          target_pm_end)).total_seconds()) if pm_end_candidates else None

    # 晚上加班：取最后一次打卡（在下午下班之后）
    overtime_candidates = [t for t in times if pm_end is None or t > pm_end]
    overtime_end = overtime_candidates[-1] if overtime_candidates else None

    # 构建结果列表，只包含非None的有效时间
    result_times = []
    if morning_start:
        result_times.append(morning_start)
    if am_end:
        result_times.append(am_end)
    if pm_start:
        result_times.append(pm_start)
    if pm_end:
        result_times.append(pm_end)
    if overtime_end and (pm_end is None or overtime_end > pm_end):
        result_times.append(overtime_end)

    return records[:4] + result_times

In [13]:
#控制显示行数，方便编写
all_rows = [list(row) for row in sheet.iter_rows(min_row=41, max_row=45, values_only=True)]

In [12]:


suc_list=[]
for i, row_data in enumerate(all_rows, 1):
    # 去除控制None
    cleaned_list = list(dropwhile(lambda x: x is None, reversed(row_data)))[::-1]
    # print(f"第{i}行数据: {cleaned_list}")
    cleaned_list=process_attendance_records(cleaned_list)

    # print(cleaned_list)

    # 处理早上上班打卡
    if isinstance(cleaned_list[4], datetime.time) and cleaned_list[4] > datetime.time(11, 0):
        # print("早上班缺卡")
        cleaned_list.insert(4, "缺卡")
        cleaned_list.insert(4, "缺卡")

    #
    # 处理早上下班打卡
    if (isinstance(cleaned_list[5], datetime.time) and
            datetime.time(11, 30) >= cleaned_list[5] > datetime.time(12, 0)):
        cleaned_list.insert(5, "缺卡")
        print(f"第{i}行修改: {cleaned_list}")
    #
    # 处理中午上班打卡
    if (isinstance(cleaned_list[6], datetime.time) and
            datetime.time(11, 35) >= cleaned_list[6] > datetime.time(13, 3)):
        cleaned_list.insert(6, "缺卡")
        # print(f"第{i}行修改: {cleaned_list}")



    print(f"第{i}行修改: {cleaned_list}")

第1行修改: ['13', '程政远', '公司', '7/12', datetime.time(7, 30), datetime.time(19, 14), datetime.time(19, 14)]
第2行修改: ['13', '程政远', '公司', '7/13', datetime.time(7, 31), datetime.time(17, 12), datetime.time(17, 12)]
第3行修改: ['13', '程政远', '公司', '7/14', '缺卡', '缺卡', datetime.time(12, 53), datetime.time(12, 53), datetime.time(19, 17)]
第4行修改: ['13', '程政远', '公司', '7/15', datetime.time(7, 31), datetime.time(19, 15), datetime.time(19, 15)]
第5行修改: ['13', '程政远', '公司', '7/16', datetime.time(7, 30), datetime.time(19, 18), datetime.time(19, 18)]


In [19]:
from datetime import time

# 原始数据
attendance_record = ['15', '胡康亮', '公司', '7/1', time(8, 3), time(17, 31)]

# 定义时间范围
start_time = time(11, 30)  # 11:30
end_time = time(12, 0)     # 12:00

# 检查所有时间记录
has_morning_punch = False
for item in attendance_record:
    if isinstance(item, time) and start_time <= item <= end_time:
        has_morning_punch = True
        break

# 如果没有打卡记录，则在第5个索引位置插入"缺卡"
if not has_morning_punch:
    # 在第5个索引位置插入（因为前4个是固定字段）
    attendance_record.insert(5, "缺卡")

print(attendance_record)

['15', '胡康亮', '公司', '7/1', datetime.time(8, 3), '缺卡', datetime.time(17, 31)]


In [22]:
import datetime

cleaned_list = ['13', '程政远', '公司', '7/10', datetime.time(7, 29), '缺卡', '缺卡', datetime.time(19, 16), datetime.time(19, 23)]
overtime = [(i, t) for i, t in enumerate(cleaned_list) if isinstance(t, datetime.time) and i >= 7]

if len(overtime) > 1:
    # 找出最晚打卡的索引
    latest_idx = max(overtime, key=lambda x: x[1])[0]
    # 删除其他加班打卡（从后往前）
    [cleaned_list.pop(i) for i, _ in sorted(overtime, reverse=True) if i != latest_idx]

print(cleaned_list)

['13', '程政远', '公司', '7/10', datetime.time(7, 29), '缺卡', '缺卡', datetime.time(19, 23)]
